In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

In [ ]:

# ChromeDriverの実際のパスを指定
chrome_driver_path = '/usr/local/bin/chromedriver'  # 実際のパスに変更

# ブラウザオプションの設定
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# ドライバーの起動
driver = webdriver.Chrome(service=Service(chrome_driver_path),
                          options=chrome_options
                          )

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import sqlite3

# Selenium WebDriver設定
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)
url = "https://www.jalan.net/jalan/doc/etc/onsenranking/onsenranking_index.html"
driver.get(url)

# ページ読み込みを待つ
time.sleep(3)

# ページソースを取得
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# データ格納用リスト
onsen_data = []

# 1〜3位を処理
top_items = soup.find_all("div", class_="areaItem", limit=3)
for rank, item in enumerate(top_items, start=1):
    try:
        # 順位は`alt`属性から取得
        rank_tag = item.find("span", class_="rank")
        if rank_tag and rank_tag.find("img"):
            rank = rank_tag.find("img")["alt"].replace("No.", "").strip()
        else:
            rank = str(rank)  # 順位が数字ならそのまま使用

        # エリア名と温泉名を取得
        area = item.find("span", class_="areaName").get_text(strip=True)
        onsen_name = item.find("h3").contents[-1].strip()  # `<h3>`内の最後のテキスト

        onsen_data.append({"順位": int(rank), "エリア名": area, "温泉名": onsen_name})
    except Exception as e:
        print(f"1〜3位エラー: {e}")

# 4位以降を処理
ranking_items = soup.find_all("div", class_=["areaItem", "areaItem3col"])
for item in ranking_items:
    try:
        # 順位を取得
        rank_tag = item.find("span", class_=["rank", "rank_g"])
        if rank_tag:
            rank = rank_tag.get_text(strip=True).replace("位", "").strip()
            if not rank.isdigit():  # 順位が数字でない場合はスキップ
                continue
            rank = int(rank)

        area = item.find("span", class_="areaName").get_text(strip=True)
        onsen_name = item.find("span", class_="areaName").find_next_sibling(string=True).strip()

        # 重複チェック
        if not any(d["エリア名"] == area and d["温泉名"] == onsen_name for d in onsen_data):
            onsen_data.append({"順位": rank, "エリア名": area, "温泉名": onsen_name})
    except Exception as e:
        print(f"4位以降エラー: {e}")

# WebDriverを終了
driver.quit()

# DataFrameに変換
df = pd.DataFrame(onsen_data)

# 順位でソート
df = df.sort_values("順位").reset_index(drop=True)

# 結果を表示
print(df)

# 必要に応じてCSVに保存
df.to_csv("onsen_ranking_fixed.csv", index=False, encoding="utf-8-sig")

# SQLiteデータベースに接続（なければ作成されます）
conn = sqlite3.connect('onsen_data.db')
c = conn.cursor()

# データベースにテーブルを作成
c.execute('''
    CREATE TABLE IF NOT EXISTS onsen (
        順位 INTEGER PRIMARY KEY,
        エリア名 TEXT,
        温泉名 TEXT
    )
''')

# DataFrameのデータをデータベースに挿入
for index, row in df.iterrows():
    c.execute('''
        INSERT OR REPLACE INTO onsen (順位, エリア名, 温泉名) VALUES (?, ?, ?)
    ''', (row['順位'], row['エリア名'], row['温泉名']))

# データベースへの保存確認
conn.commit()

# データベース接続を閉じる
conn.close()

print("データベースに保存完了")


In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import sqlite3
import pandas as pd

# Selenium WebDriver設定
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)

# タイムアウト設定を長くする
driver.set_page_load_timeout(180)

# データ取得をリトライする関数
def fetch_data_with_retry(url, retries=3, delay=5):
    for attempt in range(retries):
        try:
            driver.get(url)
            return driver.page_source
        except Exception as e:
            if attempt < retries - 1:
                print(f"再試行 {attempt + 1}/{retries} 回目... エラー: {e}")
                time.sleep(delay)
            else:
                print(f"最終試行でもエラーが発生しました: {e}")
                raise e

# URLを指定してデータを取得
url = "https://www.siken.net/j_ranking?stat=pdensity_top"
html = fetch_data_with_retry(url)

# ページ読み込み後、BeautifulSoupで解析
soup = BeautifulSoup(html, "html.parser")

# データ格納用リスト
population_density_data = []

# テーブルから人口密度を取得
table = soup.find("table")
rows = table.find_all("tr")

for row in rows[1:]:  # ヘッダーをスキップ
    try:
        cells = row.find_all("td")
        if len(cells) >= 3:
            rank = cells[0].get_text(strip=True)
            prefecture = cells[1].get_text(strip=True)
            population_density = cells[2].get_text(strip=True)

            # 結果をリストに格納
            population_density_data.append({
                "順位": int(rank.replace("位", "")),
                "都道府県": prefecture,
                "人口密度": population_density
            })
    except Exception as e:
        print(f"データ取得エラー: {e}")

# WebDriverを終了
driver.quit()

# 最後の48個のデータだけ表示
for data in population_density_data[-47:]:
    print(data)

# 必要に応じてCSVに保存
df = pd.DataFrame(population_density_data)
df.to_csv("population_density.csv", index=False, encoding="utf-8-sig")

# SQLiteデータベースに接続（なければ作成されます）
conn = sqlite3.connect('population_density.db')
c = conn.cursor()

# データベースにテーブルを作成
c.execute('''
    CREATE TABLE IF NOT EXISTS population_density (
        順位 INTEGER PRIMARY KEY,
        都道府県 TEXT,
        人口密度 TEXT
    )
''')

# データベースにデータを挿入
for index, row in df.iterrows():
    c.execute('''
        INSERT OR REPLACE INTO population_density (順位, 都道府県, 人口密度) VALUES (?, ?, ?)
    ''', (row['順位'], row['都道府県'], row['人口密度']))

# データベースへの保存確認
conn.commit()

# データベース接続を閉じる
conn.close()

print("データベースに保存完了")


In [ ]:
import pandas as pd

# データの読み込み
onsen_df = pd.read_csv("onsen_ranking_fixed.csv", encoding="utf-8-sig")
population_density_df = pd.read_csv("population_density.csv", encoding="utf-8-sig")

# 都道府県でマージ
merged_df = pd.merge(onsen_df[['順位', 'エリア名']], population_density_df[['順位', '都道府県']], left_on='エリア名', right_on='都道府県')

# 温泉の順位と人口密度の順位をそれぞれ取り出す
onsen_ranking = merged_df['順位_x']
population_density_ranking = merged_df['順位_y']

# 相関計算
correlation = onsen_ranking.corr(population_density_ranking)

# 結果を表示
print(f"温泉ランキングと人口密度ランキングの相関係数: {correlation}")
